In [1]:
from os import listdir
import numpy as np
import tensorflow as tf
import cv2
import tflite_runtime.interpreter as tflite
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, concatenate, Conv2D, MaxPooling2D,BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import time

start = time.time()

In [2]:
def loadImages(path):
    imagesList = listdir(path)
    loadedImages = []
    for image in imagesList:
        img = cv2.imread(path + image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (50,50))
        img = img.flatten()
        loadedImages.append(img)

    return loadedImages

In [3]:
path = "WSN-DS_1/test/Anomaly/"
aimgs1 = loadImages(path)
alabel1 =  np.ones(len(aimgs1))*1
path = "WSN-DS_1/test/Normal/"
nimgs1 = loadImages(path)
nlabel1 =  np.ones(len(nimgs1))*0

In [5]:
imgs1 = aimgs1.copy()
imgs1.extend(nimgs1)

In [6]:
data1 = np.array(imgs1)
data1 = data1.reshape([-1,50,50,1])
labels = np.hstack((alabel1,nlabel1))
labels =np.uint8(labels)

In [7]:
X_test_1 = (data1/255)
x = np.arange(2500).reshape((1,50,50,1))
x = np.where(x > 0, 0, x)
X_test_1 = np.concatenate((X_test_1, x))
print(X_test_1.shape)

(18734, 50, 50, 1)


In [8]:
path = "WSN-DS_2/test/Anomaly/"
aimgs2 = loadImages(path)
alabel2 =  np.ones(len(aimgs2))*1

In [9]:
path = "WSN-DS_2/test/Normal/"
nimgs2 = loadImages(path)
nlabel2 =  np.ones(len(nimgs2))*0

In [10]:
imgs2 = aimgs2.copy()
imgs2.extend(nimgs2)

In [11]:
data2 = np.array(imgs2)
data2 = data2.reshape([-1,50,50,1])
labels = np.hstack((alabel2,nlabel2))
labels =np.uint8(labels)

In [12]:
X_test_2 = data2/255
print(X_test_2.shape)

(18734, 50, 50, 1)


In [13]:
y_test = to_categorical(labels)

In [20]:
model = load_model('fusion_light_weight_model.tflite')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_input (InputLayer)       [(None, 50, 50, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_4_input (InputLayer)     [(None, 50, 50, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 46, 46, 32)   832         conv2d_input[0][0]               
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 46, 46, 32)   832         conv2d_4_input[0][0]             
______________________________________________________________________________________________

In [15]:
y_pred = model.predict([X_test_1, X_test_2 ])

In [16]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test,y_pred.round()))

0.973417316109747


In [17]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test,y_pred.round()))\\

              precision    recall  f1-score   support

           0       0.97      1.00      0.99     16864
           1       1.00      0.73      0.85      1870

   micro avg       0.97      0.97      0.97     18734
   macro avg       0.99      0.87      0.92     18734
weighted avg       0.97      0.97      0.97     18734
 samples avg       0.97      0.97      0.97     18734



In [18]:
confusion_matrix(y_test.argmax(axis=1), y_pred.round().argmax(axis=1))

array([[16864,     0],
       [  498,  1372]], dtype=int64)

In [19]:
print("Total time: ", time.time() - start, "seconds")

Total time:  219.97496438026428 seconds
